In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
def consumer():
    r = ''
    print('Start consumer')
    while True:
        n = yield r #c.send(None)在这里停止，接下来把1赋值给n
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    print('flag0')
    c.send(None)
    print('flag1')
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c=consumer()
produce(c)

flag0
Start consumer
flag1
[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


In [39]:
def h():
    print ('Wen Chuan')
    m = yield ''  # Fighting!
    print (m)
    d = yield 12
    print ('We are together!')
    return '12'

c = h()
#c.send(None)  #如果不send(none)，就没有yield来接受这个值
#c.send('Fighting!')  #(yield 5)表达式被赋予了'Fighting!'


<generator object middle at 0x10350c830>

http://www.cnblogs.com/coderzh/articles/1202040.html

# throw()

In [10]:
def gen_func():
    try:
        yield 1
    except Exception as e:
        pass
    yield 2
    yield 3
    
if __name__=='__main__':
    gen=gen_func()
    print(next(gen))
    gen.throw(Exception,'download error')

1


2

# yield from

In [ ]:
yield from iterable
类似于
for value in iterable:
    yield value

In [ ]:
## 通过middle 在子生成器（sales_sum）与main函数中直接建立链接

In [1]:
final_result={}
def sales_sum(products):
    total=0
    nums=[]
    while True:
        x=yield
        print(products+'销量',x)
        if not x:
            break
        total +=x
        nums.append(x)
    return total,nums

def middle(key):
    #这个循环每次都会新建一个sales_sum实例，每个实例都是作为协程使用的生成器对象
    while True:
        print(1)
        final_result[key]=yield from sales_sum(key) # yield from处理子生成器产生的 stopiteration 并捕获result
        print(key+'销量统计完成')
    
def main():
    data_sets={
        "bobby牌面膜":[1200,1500,3000],
        "bobby牌手机":[28,55,98,108],
        "bobby牌大衣":[280,560,778,70]
    }
    for key, data_set in data_sets.items():
        print("start key:", key)
        m=middle(key)
        m.send(None)    # 激活协程m
        for value in data_set:
            # 把各个value传给middle 传入的值最终到达sales_sum函数中；
            # middle并不知道传入的是什么，同时middle实例,m在yield from处暂停
            m.send(value)
        m.send(None)
        print("final_results", final_result)

main()

start key: bobby牌面膜
1
bobby牌面膜销量 1200
bobby牌面膜销量 1500
bobby牌面膜销量 3000
bobby牌面膜销量 None
bobby牌面膜销量统计完成
1
final_results {'bobby牌面膜': (5700, [1200, 1500, 3000])}
start key: bobby牌手机
1
bobby牌手机销量 28
bobby牌手机销量 55
bobby牌手机销量 98
bobby牌手机销量 108
bobby牌手机销量 None
bobby牌手机销量统计完成
1
final_results {'bobby牌面膜': (5700, [1200, 1500, 3000]), 'bobby牌手机': (289, [28, 55, 98, 108])}
start key: bobby牌大衣
1
bobby牌大衣销量 280
bobby牌大衣销量 560
bobby牌大衣销量 778
bobby牌大衣销量 70
bobby牌大衣销量 None
bobby牌大衣销量统计完成
1
final_results {'bobby牌面膜': (5700, [1200, 1500, 3000]), 'bobby牌手机': (289, [28, 55, 98, 108]), 'bobby牌大衣': (1688, [280, 560, 778, 70])}


grouper 发送的每个值都会经由 yield from 处理，通过管道传给 averager 实例。grouper 会在 yield from 表达式处暂停，等待 averager 实例处理客户端发来的值。averager 实例运行完毕后，返回的值绑定到 results[key] 上。while 循环会不断创建 averager 实例，处理更多的值。

外层 for 循环重新迭代时会新建一个 grouper 实例，然后绑定到 group 变量上。前一个 grouper 实例（以及它创建的尚未终止的 averager 子生成器实例）被垃圾回收程序回收。
https://www.cnblogs.com/gqtcgq/p/8126124.html